In [2]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset, DatasetDict
from tqdm import tqdm

2024-09-30 15:11:12.757968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 15:11:12.767545: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 15:11:12.770674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 15:11:12.778465: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 15:11:13.487511: W tensorflow/compiler/tf2

In [5]:
ds = load_dataset("jlbaker361/wikiart")

In [6]:
# constants
HEIGHT = 64
WIDTH = 64
CHANNELS = 3
BATCH_SIZE = 32

# Image Size Normalization, Augumentation


In [33]:
n_obs = 5000
train_ds = ds['train'].select(range(n_obs))
def convert_img(x):
    import numpy as np  #subprocesses lmao
    HEIGHT = 64
    WIDTH = 64
    x['img_pixels'] = np.array(x['image'].resize((WIDTH, HEIGHT))).reshape(-1)/255
    return x

train_ds = train_ds.map(convert_img, num_proc=4)
train_ds = train_ds.filter(lambda x: len(x['img_pixels']) == HEIGHT*WIDTH*3)

Map (num_proc=4):   6%|▋         | 4605/73304 [00:05<02:00, 569.15 examples/s] 

In [10]:
# one-hot-encoding
# get unique class names

def get_unique_styles(train_ds: DatasetDict):
    unique_style_list = []
    for x in tqdm(train_ds):
        if x['style'] not in unique_style_list:
            unique_style_list.append(x['style'])
    return unique_style_list

def style2idx(style, style_list):
    return style_list.index(style)

unique_styles = get_unique_styles(train_ds)
unique_styles


100%|██████████| 4989/4989 [00:16<00:00, 294.68it/s]


['impressionism',
 'mannerism-late-renaissance',
 'expressionism',
 'post-impressionism',
 'early-renaissance',
 'realism',
 'fauvism',
 'romanticism',
 'art-nouveau-modern',
 'symbolism',
 'pop-art',
 'baroque',
 'abstract-expressionism',
 'pointillism',
 'na-ve-art-primitivism',
 'rococo',
 'northern-renaissance',
 'cubism',
 'minimalism',
 'synthetic-cubism',
 'high-renaissance',
 'ukiyo-e',
 'contemporary-realism',
 'color-field-painting',
 'analytical-cubism',
 'new-realism',
 'action-painting']

In [27]:
# for cnn
def encode_labels_cnn(x, unique_labels):
    x['y'] = np.zeros(len(unique_labels), dtype=np.int32)
    x['y'][style2idx(x['style'], unique_labels)] = 1
    return x

# for other clfs
def encode_labels(x, unique_labels):
    x['y'] = style2idx(x['style'], unique_labels)
    return x

train_ds = train_ds.map(lambda x: encode_labels(x, unique_styles), num_proc=4)

Map (num_proc=4): 100%|██████████| 4989/4989 [00:00<00:00, 8616.69 examples/s]


0

In [32]:
processed_ds = train_ds.remove_columns(['image', 'text', 'name', 'gen_style'])
SAVE_DIR = '../processed_data'
processed_ds.save_to_disk(SAVE_DIR)


Saving the dataset (1/1 shards): 100%|██████████| 4989/4989 [00:00<00:00, 9157.70 examples/s] 


In [26]:
processed_ds

Dataset({
    features: ['style', 'img_pixels', 'y'],
    num_rows: 4989
})

In [16]:
def augment_with_tf(image):
    image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image


Map:   0%|          | 0/4989 [00:00<?, ? examples/s]

In [ ]:
def preprocess_and_augment(examples):
    examples['img_pixels'] = [augment_with_tf(img) for img in examples['img_pixels']]
    return examples

In [ ]:
train_ds_augmented = train_ds.map(preprocess_and_augment, batched=True)

In [ ]:
def preprocess_batch(ds, batch_size=BATCH_SIZE):
    def generator():
        for row in ds:
            yield row['img_pixels'], row['label']
    dataset = tf.data.Dataset.from_generator(
        generator, 
        output_signature=(
            tf.TensorSpec(shape=(HEIGHT, WIDTH, CHANNELS), dtype=tf.float32),  # Image shape
            tf.TensorSpec(shape=(), dtype=tf.int64)  # Label shape
        )
    )
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)
    return dataset

In [ ]:
train_dataset = preprocess_batch(train_ds_augmented)

In [17]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>